# 4. Improving and understanding algorithm performance.

Having built a basic machine learning pipeline, one wants to improve performance to get the best results, and also understand the results of the trained algorithm.

## Overview 

### Prerequisites

### Learning Outcomes 
* Understand how to improve results through hyperparameter tuning
* Understand basic approaches to improving training data through resampling approaches
* Understand basic explainability and interpretability methods
* Understand the importance of experiment tracking and how to do it
* Understand where are perfoprmance bottlenecks and strategies for performance optimisation


## Tutorial 
Hyperparameter Tuning

Data Augmentation
 - imbalance learn
 - resampling classes
 

 
Interpretability & explainability
https://github.com/djgagne/ams-ml-python-course module 4 deals with interpretability

Feature Importance

Keeping track of results - ML Flow 
Notebook 4 - improving results

Hyperparameters tuning
Making better use of your data
Explainability and interpretability

Next steps 
Data duration and cataloging 
Scaling
Experiment tracking
Saving models



## Excercise - Hyperparameter tuning


In [2]:
import pathlib
import os
import functools
import math
import datetime

In [3]:
import pandas
import numpy

In [4]:
import matplotlib
import matplotlib.pyplot
%matplotlib inline

In [15]:
import sklearn
import sklearn.tree
import sklearn.preprocessing
import sklearn.ensemble

In [5]:
try:
    falklands_data_dir = os.environ['OPMET_ROTORS_DATA_ROOT']
except KeyError:
    falklands_data_dir = '/project/informatics_lab/data_science_cop/ML_challenges/2021_opmet_challenge'
falklands_data_dir = pathlib.Path(falklands_data_dir) /  'Rotors'

In [6]:
falklands_data_fname = 'new_training.csv'
falklands_data_path = falklands_data_dir / falklands_data_fname
falklands_df = pandas.read_csv(falklands_data_path)

In [7]:
temp_feature_names = [f'air_temp_{i1}' for i1 in range(1,23)]
humidity_feature_names = [f'sh_{i1}' for i1 in range(1,23)]
wind_direction_feature_names = [f'winddir_{i1}' for i1 in range(1,23)]
wind_speed_feature_names = [f'windspd_{i1}' for i1 in range(1,23)]
target_feature_name = 'rotors_present'


In [8]:
falklands_df = falklands_df.rename({'Rotors 1 is true': target_feature_name},axis=1)
falklands_df.loc[falklands_df[falklands_df[target_feature_name].isna()].index, target_feature_name] = 0
falklands_df['DTG'] = pandas.to_datetime(falklands_df['DTG'])
falklands_df = falklands_df.drop_duplicates(subset=['DTG'])
falklands_df = falklands_df[~falklands_df['DTG'].isnull()]
falklands_df = falklands_df[(falklands_df['wind_speed_obs'] >= 0.0) &
                            (falklands_df['air_temp_obs'] >= 0.0) &
                            (falklands_df['wind_direction_obs'] >= 0.0) &
                            (falklands_df['dewpoint_obs'] >= 0.0) 
                           ]
falklands_df = falklands_df.drop_duplicates(subset='DTG')
falklands_df[target_feature_name]  = falklands_df[target_feature_name] .astype(bool)
falklands_df['time'] = pandas.to_datetime(falklands_df['DTG'])

In [9]:
def get_v_wind(wind_dir_name, wind_speed_name, row1):
    return math.cos(math.radians(row1[wind_dir_name])) * row1[wind_speed_name]

def get_u_wind(wind_dir_name, wind_speed_name, row1):
    return math.sin(math.radians(row1[wind_dir_name])) * row1[wind_speed_name]

In [10]:
%%time
u_feature_template = 'u_wind_{level_ix}'
v_feature_template = 'v_wind_{level_ix}'
u_wind_feature_names = []
v_wind_features_names = []
for wsn1, wdn1 in zip(wind_speed_feature_names, wind_direction_feature_names):
    level_ix = int( wsn1.split('_')[1])
    u_feature = u_feature_template.format(level_ix=level_ix)
    u_wind_feature_names += [u_feature]
    falklands_df[u_feature] = falklands_df.apply(functools.partial(get_u_wind, wdn1, wsn1), axis='columns')
    v_feature = v_feature_template.format(level_ix=level_ix)
    v_wind_features_names += [v_feature]
    falklands_df[v_feature] = falklands_df.apply(functools.partial(get_v_wind, wdn1, wsn1), axis='columns')

CPU times: user 16.3 s, sys: 1.66 s, total: 18 s
Wall time: 18 s


In [11]:
rotors_train_df = falklands_df[falklands_df['time'] < datetime.datetime(2020,1,1,0,0)]
rotors_test_df = falklands_df[falklands_df['time'] > datetime.datetime(2020,1,1,0,0)]

In [12]:
def preproc_input(data_subset, pp_dict):
    return numpy.concatenate([scaler1.transform(data_subset[[if1]]) for if1,scaler1 in pp_dict.items()],axis=1)

def preproc_target(data_subset, enc1):
     return enc1.transform(data_subset[[target_feature_name]])


In [16]:
input_feature_names = temp_feature_names + humidity_feature_names + u_wind_feature_names + v_wind_features_names
preproc_dict = {}
for if1 in input_feature_names:
    scaler1 = sklearn.preprocessing.StandardScaler()
    scaler1.fit(rotors_train_df[[if1]])
    preproc_dict[if1] = scaler1
    
target_encoder = sklearn.preprocessing.LabelEncoder()
target_encoder.fit(rotors_train_df[[target_feature_name]])

/home/h01/shaddad/.conda/envs/ml-weather-tutorial-tf/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LabelEncoder()

In [17]:
X_train_rotors = preproc_input(rotors_train_df, preproc_dict)
y_train_rotors = preproc_target(rotors_train_df, target_encoder)

/home/h01/shaddad/.conda/envs/ml-weather-tutorial-tf/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [18]:
X_test_rotors = preproc_input(rotors_test_df, preproc_dict)
y_test_rotors = preproc_target(rotors_test_df, target_encoder)

/home/h01/shaddad/.conda/envs/ml-weather-tutorial-tf/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [19]:
sklearn.tree.DecisionTreeClassifier().get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [54]:
clf_opts = {'max_depth':[5,10,15,20], 
            'min_samples_leaf': [1,2,5],
            'min_samples_split': [4,10,20],
            'ccp_alpha': [0.0, 0.01, 0.1],
           }


In [ ]:
%%time
clf1 = sklearn.tree.DecisionTreeClassifier()
cv1 = sklearn.model_selection.KFold(n_splits=5, shuffle=True)
hpt_grid = sklearn.model_selection.GridSearchCV(estimator=clf1, 
                                                param_grid=clf_opts,
                                                cv=cv1,
                                               )
res1 = hpt_grid.fit(X_train_rotors, y_train_rotors)

In [53]:
hpt_grid.best_estimator_

DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, min_samples_split=20)

In [51]:
import scipy.stats

In [52]:
%%time
clf1 = sklearn.tree.DecisionTreeClassifier()
cv1 = sklearn.model_selection.KFold(n_splits=5, shuffle=True)
hpt_random = sklearn.model_selection.RandomizedSearchCV(estimator=clf1,
                                                        param_distributions={
                                                            'max_depth': scipy.stats.randint(5,10), 
                                                            'min_samples_leaf': scipy.stats.randint(1,7),
                                                            'min_samples_split': scipy.stats.randint(4,20),
                                                        },
                                                        cv=cv1,
                                                        n_iter=10,
                                                     )
res1 = hpt_random.fit(X_train_rotors, y_train_rotors)  

CPU times: user 33.6 s, sys: 2 ms, total: 33.6 s
Wall time: 33.6 s


In [ ]:
hpt_random

## Example - resampling data for imbalanced classes

In [38]:
no_rotors_df = rotors_train_df[rotors_train_df[target_feature_name] == False]
rotors_df = rotors_train_df[rotors_train_df[target_feature_name] == True]

In [39]:
num_resamples = 5000

In [40]:
rotors_resampled_train = pandas.concat([
    no_rotors_df.sample(num_resamples, replace=True),
    rotors_df.sample(num_resamples, replace=True),
])

In [41]:
X_train_resampled_rotors = preproc_input(rotors_resampled_train, preproc_dict)
y_train_resampled_rotors = preproc_target(rotors_resampled_train, target_encoder)

/home/h01/shaddad/.conda/envs/ml-weather-tutorial-tf/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


## Example - Feature Importance

In [ ]:
import sklearn.inspection

In [ ]:
permutation_importance

## Examples of use
*UNDER  CONSTRUCTION*


## Next steps

https://github.com/djgagne/ams-ml-python-course

Interpretable/Explainable Machine Learning - Theory
* https://christophm.github.io/interpretable-ml-book/limo.html
* https://github.com/interpretml/interpret

Interpretable/Explainable Machine Learning - Practical Examples
Colab - What IF Tool (WIT)
* https://pair-code.github.io/what-if-tool/
* https://colab.research.google.com/github/pair-code/what-if-tool/blob/master/WIT_COMPAS_with_SHAP.ipynb#scrollTo=iywHwbJJkeYG

* https://github.com/neomatrix369/awesome-ai-ml-dl/blob/master/data/model-analysis-interpretation-explainability.md

Interpretable/Explainable Machine Learning - Reading (with examples)
* https://towardsdatascience.com/deep-learning-model-interpretation-using-shap-a21786e91d16
* https://towardsdatascience.com/explain-any-models-with-the-shap-values-use-the-kernelexplainer-79de9464897a
* https://medium.com/dataman-in-ai/explain-your-model-with-lime-5a1a5867b423
* https://medium.com/analytics-vidhya/explain-your-model-with-microsofts-interpretml-5daab1d693b4
* https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d
* https://medium.com/dataman-in-ai/identify-causality-by-fixed-effects-model-585554bd9735
* https://medium.com/dataman-in-ai/identify-causality-by-difference-in-differences-78ad8335fb7c
* https://medium.com/dataman-in-ai/identify-causality-by-regression-discontinuity-a4c8fb7507df

XAI Short Course Full Syllabus
* https://github.com/thunderhoser/ai2es_xai_course
* https://www.youtube.com/playlist?list=PLbelYhZAAHEIr4iC1FNcPXUUYXI0zg_96
* https://www.youtube.com/watch?v=DfqSZu3aS8E


## Dataset Info
*UNDER  CONSTRUCTION*


## References
* 